In [77]:
import pandas as pd
import numpy as np
import scipy as sc
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn import linear_model
from sklearn import mixture

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

import os
import sys
import math

sys._enablelegacywindowsfsencoding()

init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

## Подготовка данных

In [2]:
df = pd.read_csv('output.csv', sep=';', encoding='utf-8')

df = df.drop(df[df.vertical_force_mean > 300].index)
df = df.drop(df[df.edge_external_force_mean > 250].index)
df = df.drop(df[df.edge_internal_force_mean > 250].index)
df = df.drop(df[df.lateral_plus_force_mean > 250].index)
df = df.drop(df[df.lateral_minus_force_mean < -250].index)

df = df.drop(df.columns[0], axis=1)

df = pd.get_dummies(df, columns=['season'])
df = pd.get_dummies(df, columns=['weight'])
df = pd.get_dummies(df, columns=['type_area'])
df = pd.get_dummies(df, columns=['type_clamping'])

df = df.rename(index=str, columns={'season_Весна': 'spring', 'season_Лето': 'summer', 'season_Осень': 'fall', 'weight_23.5': '23.5', 'weight_27.0': '27', 'type_area_Р-350': '350', 'type_area_Р-650': '650', 'type_area_прямая': '0', 'type_clamping_АРС': 'ars', 'type_clamping_ДО': 'du', 'type_clamping_КБ': 'kb'})

df.vertical_force_mean = df.vertical_force_mean * 1000
df.edge_external_force_mean = df.edge_external_force_mean * 1000
df.edge_internal_force_mean = df.edge_internal_force_mean * 1000
df.lateral_minus_force_mean = df.lateral_minus_force_mean * 1000
df.lateral_plus_force_mean = df.lateral_plus_force_mean * 1000
df.speed = df.speed * 1000 / 3600

temp = pd.read_csv('Weather.txt', sep=':', encoding='utf-8')
temp = temp.replace(' ','',regex=True)
df = pd.merge(df, temp, left_on='date', right_on='date ', how='outer')
df = df.drop('date ', axis=1)


df.temperature = df.temperature.astype('int32')

In [3]:
df_ars = df.loc[df.ars == 1]
df_du = df.loc[df.du == 1]
df_kb = df.loc[df.kb == 1]

df_ars_23_0 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_ars_23_350 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_ars_23_650 = df.loc[(df.ars == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_ars_27_0 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_ars_27_350 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_ars_27_650 = df.loc[(df.ars == 1) & (df['27'] == 1) & (df['650'] == 1)]

df_du_23_0 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_du_23_350 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_du_23_650 = df.loc[(df.du == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_du_27_0 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_du_27_350 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_du_27_650 = df.loc[(df.du == 1) & (df['27'] == 1) & (df['650'] == 1)]

df_kb_23_0 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['0'] == 1)]
df_kb_23_350 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['350'] == 1)]
df_kb_23_650 = df.loc[(df.kb == 1) & (df['23.5'] == 1) & (df['650'] == 1)]
df_kb_27_0 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['0'] == 1)]
df_kb_27_350 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['350'] == 1)]
df_kb_27_650 = df.loc[(df.kb == 1) & (df['27'] == 1) & (df['650'] == 1)]

In [4]:
df_list = [df_ars_23_0, df_ars_23_650, df_ars_27_0, df_ars_27_650, df_du_23_0, df_du_23_650, df_du_27_0, df_du_27_650, df_kb_23_0, df_kb_23_650, df_kb_27_0, df_kb_27_650]
name_list = ['АРС, 23.5 тонн/ось, прямая',
             'АРС, 23.5 тонн/ось, радиус 650',
             'АРС, 27 тонн/ось, прямая',
             'АРС, 27 тонн/ось, радиус 650',
             'ДУ, 23.5 тонн/ось, прямая',
             'ДУ, 23.5 тонн/ось, радиус 650',
             'ДУ, 27 тонн/ось, прямая',
             'ДУ, 27 тонн/ось, радиус 650',
             'КБ, 23.5 тонн/ось, прямая',
             'КБ, 23.5 тонн/ось, радиус 650',
             'КБ, 27 тонн/ось, прямая',
             'КБ, 27 тонн/ось, радиус 650']

## Корелляционный анализ

Генеральная совокупность

In [41]:
trace = go.Table(
    header=dict(values=list(['Имя', 'Температура', 'Скорость']),
                fill = dict(color='#C2D4FF')),
    cells=dict(values=[['Внешняя кромочная',
                       'Внутренняя кромочная',
                       'Боковая отрицательная',
                       'Боковая положительная',
                       'Вертикальная'],
                       [round(df.corr('spearman').temperature.loc['edge_external_force_mean'], 3),
                                        round(df.corr('spearman').temperature.loc['edge_internal_force_mean'], 3),
                                        round(df.corr('spearman').temperature.loc['lateral_minus_force_mean'], 3),
                                        round(df.corr('spearman').temperature.loc['lateral_plus_force_mean'], 3),
                                        round(df.corr('spearman').temperature.loc['vertical_force_mean'], 3)],
                       [round(df.corr('spearman').speed.loc['edge_external_force_mean'], 3),
                                        round(df.corr('spearman').speed.loc['edge_internal_force_mean'], 3),
                                        round(df.corr('spearman').speed.loc['lateral_minus_force_mean'], 3),
                                        round(df.corr('spearman').speed.loc['lateral_plus_force_mean'], 3),
                                        round(df.corr('spearman').speed.loc['vertical_force_mean'], 3)]],
               fill = dict(color='#F5F8FF')))

data = [trace]

layout = dict(title = 'Генеральная совокупность')

fig = dict(data=data, layout=layout)
iplot(fig)

In [46]:
df_tmp = df_ars

trace = go.Table(
    header=dict(values=list(['Имя', 'Температура', 'Скорость']),
                fill = dict(color='#C2D4FF')),
    cells=dict(values=[['Внешняя кромочная',
                       'Внутренняя кромочная',
                       'Боковая отрицательная',
                       'Боковая положительная',
                       'Вертикальная'],
                       [round(df_tmp.corr('spearman').temperature.loc['edge_external_force_mean'], 3),
                                        round(df_tmp.corr('spearman').temperature.loc['edge_internal_force_mean'], 3),
                                        round(df_tmp.corr('spearman').temperature.loc['lateral_minus_force_mean'], 3),
                                        round(df_tmp.corr('spearman').temperature.loc['lateral_plus_force_mean'], 3),
                                        round(df_tmp.corr('spearman').temperature.loc['vertical_force_mean'], 3)],
                       [round(df.corr('spearman').speed.loc['edge_external_force_mean'], 3),
                                        round(df_tmp.corr('spearman').speed.loc['edge_internal_force_mean'], 3),
                                        round(df_tmp.corr('spearman').speed.loc['lateral_minus_force_mean'], 3),
                                        round(df_tmp.corr('spearman').speed.loc['lateral_plus_force_mean'], 3),
                                        round(df_tmp.corr('spearman').speed.loc['vertical_force_mean'], 3)]],
               fill = dict(color='#F5F8FF')))

data = [trace]

layout = dict(title = 'Генеральная совокупность для АРС')

fig = dict(data=data, layout=layout)
iplot(fig)

In [45]:
i = 0
for df_tmp in df_list:
    trace = go.Table(
        header=dict(values=list(['Имя', 'Температура', 'Скорость']),
                    fill = dict(color='#C2D4FF')),
        cells=dict(values=[['Внешняя кромочная',
                           'Внутренняя кромочная',
                           'Боковая отрицательная',
                           'Боковая положительная',
                           'Вертикальная'],
                           [round(df_tmp.corr('spearman').temperature.loc['edge_external_force_mean'], 3),
                                            round(df_tmp.corr('spearman').temperature.loc['edge_internal_force_mean'], 3),
                                            round(df_tmp.corr('spearman').temperature.loc['lateral_minus_force_mean'], 3),
                                            round(df_tmp.corr('spearman').temperature.loc['lateral_plus_force_mean'], 3),
                                            round(df_tmp.corr('spearman').temperature.loc['vertical_force_mean'], 3)],
                           [round(df_tmp.corr('spearman').speed.loc['edge_external_force_mean'], 3),
                                            round(df_tmp.corr('spearman').speed.loc['edge_internal_force_mean'], 3),
                                            round(df_tmp.corr('spearman').speed.loc['lateral_minus_force_mean'], 3),
                                            round(df_tmp.corr('spearman').speed.loc['lateral_plus_force_mean'], 3),
                                            round(df_tmp.corr('spearman').speed.loc['vertical_force_mean'], 3)]],
                   fill = dict(color='#F5F8FF')))

    data = [trace]

    layout = dict(title = name_list[i])
    i = i + 1

    fig = dict(data=data, layout=layout)
    iplot(fig)

## Статистические критерии

Проверим, как на силы влияет тип скрепления. Используем критерий Краскела — Уоллиса

https://en.wikipedia.org/wiki/Kruskal–Wallis_one-way_analysis_of_variance

Масса 23.5 тонн/ось, кривизна 0, лето

In [78]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].vertical_force_mean)

KruskalResult(statistic=3.0375442739079057, pvalue=0.21898060013366696)

In [118]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=4.008791208791209, pvalue=0.13474170838656888)

In [119]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=0.33532182103611063, pvalue=0.8456405338841163)

In [120]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=4.611616954474094, pvalue=0.09967818052825451)

In [121]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.summer == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.summer == 1].edge_external_force_mean)

KruskalResult(statistic=6.103610675039249, pvalue=0.04727350267876055)

Масса 23.5 тонн/ось, кривизна 0, весна

In [79]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].vertical_force_mean)

KruskalResult(statistic=8.377894736842109, pvalue=0.015162236725046708)

In [122]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].lateral_plus_force_mean)

KruskalResult(statistic=1.2463157894736838, pvalue=0.5362483451366988)

In [123]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].lateral_minus_force_mean)

KruskalResult(statistic=0.10985915492957887, pvalue=0.9465518041841647)

In [124]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].edge_internal_force_mean)

KruskalResult(statistic=3.7252631578947373, pvalue=0.15526350411136827)

In [125]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.spring == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.spring == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.spring == 1].edge_external_force_mean)

KruskalResult(statistic=13.3042105263158, pvalue=0.0012913007140293988)

Масса 23.5 тонн/ось, кривизна 0, осень

In [80]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].vertical_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].vertical_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].vertical_force_mean)

KruskalResult(statistic=12.622448979591837, pvalue=0.0018158084384611104)

In [126]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].lateral_plus_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].lateral_plus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].lateral_plus_force_mean)

KruskalResult(statistic=2.263163265306119, pvalue=0.322522740306774)

In [127]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].lateral_minus_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].lateral_minus_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].lateral_minus_force_mean)

KruskalResult(statistic=4.8367346938775455, pvalue=0.08906691418764005)

In [128]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].edge_internal_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].edge_internal_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].edge_internal_force_mean)

KruskalResult(statistic=10.34693877551021, pvalue=0.005664881017821615)

In [129]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.fall == 1].edge_external_force_mean,
                df_du_23_0.loc[df_du_23_0.fall == 1].edge_external_force_mean,
                df_kb_23_0.loc[df_kb_23_0.fall == 1].edge_external_force_mean)

KruskalResult(statistic=9.857142857142861, pvalue=0.007236834234988313)

Масса 23.5 тонн/ось, кривизна 650, лето

In [84]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].vertical_force_mean)

KruskalResult(statistic=12.426470588235304, pvalue=0.002002747494063425)

In [71]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].lateral_plus_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].lateral_plus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=7.123809523809527, pvalue=0.028384707081217604)

In [72]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].lateral_minus_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].lateral_minus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=9.257142857142853, pvalue=0.00976870443106656)

In [73]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].edge_external_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].edge_external_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].edge_external_force_mean)

KruskalResult(statistic=6.074285714285708, pvalue=0.047971756037566)

In [74]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.summer == 1].edge_internal_force_mean,
                df_du_23_650.loc[df_du_23_650.summer == 1].edge_internal_force_mean,
                df_kb_23_650.loc[df_kb_23_650.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=6.935238095238091, pvalue=0.031191207099543674)

Масса 23.5 тонн/ось, кривизна 650, весна

In [117]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].vertical_force_mean)

KruskalResult(statistic=14.019607843137251, pvalue=0.0009029856270441209)

In [118]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].lateral_plus_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].lateral_plus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].lateral_plus_force_mean)

KruskalResult(statistic=2.1788048552754447, pvalue=0.33641746743330614)

In [119]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].lateral_minus_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].lateral_minus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].lateral_minus_force_mean)

ValueError: All numbers are identical in kruskal

In [52]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].edge_external_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].edge_external_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].edge_external_force_mean)

KruskalResult(statistic=11.666666666666671, pvalue=0.0029282996948181788)

In [53]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.spring == 1].edge_internal_force_mean,
                df_du_23_650.loc[df_du_23_650.spring == 1].edge_internal_force_mean,
                df_kb_23_650.loc[df_kb_23_650.spring == 1].edge_internal_force_mean)

KruskalResult(statistic=10.967320261437905, pvalue=0.004154097299489192)

Масса 23.5 тонн/ось, кривизна 650, осень

In [85]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].vertical_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].vertical_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].vertical_force_mean)

KruskalResult(statistic=11.200000000000003, pvalue=0.0036978637164829255)

In [47]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].lateral_plus_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].lateral_plus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].lateral_plus_force_mean)

KruskalResult(statistic=7.342857142857149, pvalue=0.025440100973637338)

In [48]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].lateral_minus_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].lateral_minus_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].lateral_minus_force_mean)

KruskalResult(statistic=10.628571428571433, pvalue=0.004920792352021811)

In [49]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].edge_external_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].edge_external_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].edge_external_force_mean)

KruskalResult(statistic=5.485714285714295, pvalue=0.06438612348937231)

In [50]:
sc.stats.kruskal(df_ars_23_650.loc[df_ars_23_650.fall == 1].edge_internal_force_mean,
                df_du_23_650.loc[df_du_23_650.fall == 1].edge_internal_force_mean,
                df_kb_23_650.loc[df_kb_23_650.fall == 1].edge_internal_force_mean)

KruskalResult(statistic=10.857142857142861, pvalue=0.0043893618427784335)

Масса 27 тонн/ось, кривизна 0, лето

In [92]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].vertical_force_mean)

KruskalResult(statistic=6.077922077922082, pvalue=0.04788461390707669)

In [54]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].lateral_plus_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].lateral_plus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=14.81576086956521, pvalue=0.0006064547536325507)

In [55]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].lateral_minus_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].lateral_minus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=11.362275449101798, pvalue=0.0034096769625951385)

In [56]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].edge_external_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].edge_external_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].edge_external_force_mean)

KruskalResult(statistic=4.876630434782598, pvalue=0.08730782232089292)

In [57]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.summer == 1].edge_internal_force_mean,
                df_du_27_0.loc[df_du_27_0.summer == 1].edge_internal_force_mean,
                df_kb_27_0.loc[df_kb_27_0.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=12.704347826086945, pvalue=0.001742953984362984)

Масса 27 тонн/ось, кривизна 0, весна

In [93]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.spring == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.spring == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.spring == 1].vertical_force_mean)

KruskalResult(statistic=16.102500000000006, pvalue=0.00031870329427263614)

In [58]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.spring == 1].lateral_plus_force_mean,
                df_du_27_0.loc[df_du_27_0.spring == 1].lateral_plus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.spring == 1].lateral_plus_force_mean)

KruskalResult(statistic=8.676363636363646, pvalue=0.013060252543035196)

In [59]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.spring == 1].lateral_minus_force_mean,
                df_du_27_0.loc[df_du_27_0.spring == 1].lateral_minus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.spring == 1].lateral_minus_force_mean)

KruskalResult(statistic=3.1275598086124425, pvalue=0.20934327624879984)

In [60]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.spring == 1].edge_external_force_mean,
                df_du_27_0.loc[df_du_27_0.spring == 1].edge_external_force_mean,
                df_kb_27_0.loc[df_kb_27_0.spring == 1].edge_external_force_mean)

KruskalResult(statistic=16.305772727272725, pvalue=0.0002879031659701535)

Масса 27 тонн/ось, кривизна 0, осень

In [94]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].vertical_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].vertical_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].vertical_force_mean)

KruskalResult(statistic=14.3625730994152, pvalue=0.0007606885487011675)

In [61]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].lateral_plus_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].lateral_plus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].lateral_plus_force_mean)

KruskalResult(statistic=6.467836257309941, pvalue=0.03940281066076385)

In [64]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].lateral_minus_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].lateral_minus_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].lateral_minus_force_mean)

KruskalResult(statistic=5.192982456140349, pvalue=0.0745346450047699)

In [62]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].edge_external_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].edge_external_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].edge_external_force_mean)

KruskalResult(statistic=12.292397660818722, pvalue=0.0021416069288341354)

In [63]:
sc.stats.kruskal(df_ars_27_0.loc[df_ars_27_0.fall == 1].edge_internal_force_mean,
                df_du_27_0.loc[df_du_27_0.fall == 1].edge_internal_force_mean,
                df_kb_27_0.loc[df_kb_27_0.fall == 1].edge_internal_force_mean)

KruskalResult(statistic=14.3625730994152, pvalue=0.0007606885487011675)

Масса 27 тонн/ось, кривизна 650, лето

In [95]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].vertical_force_mean)

KruskalResult(statistic=3.676457680250792, pvalue=0.1590989664122756)

In [65]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].lateral_plus_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].lateral_plus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=11.031339031339044, pvalue=0.004023232819134385)

In [66]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].lateral_minus_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].lateral_minus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=7.4689033436037935, pvalue=0.023886264991177537)

In [67]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].edge_external_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].edge_external_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].edge_external_force_mean)

KruskalResult(statistic=0.7284541723666398, pvalue=0.6947334124694389)

In [68]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.summer == 1].edge_internal_force_mean,
                df_du_27_650.loc[df_du_27_650.summer == 1].edge_internal_force_mean,
                df_kb_27_650.loc[df_kb_27_650.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=3.4641472868217043, pvalue=0.1769171662191844)

Масса 27 тонн/ось, кривизна 650, весна

In [99]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].vertical_force_mean)

KruskalResult(statistic=14.921052631578945, pvalue=0.0005753532742717005)

In [69]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].lateral_plus_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].lateral_plus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].lateral_plus_force_mean)

KruskalResult(statistic=9.705882352941181, pvalue=0.007805386735968123)

In [70]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].lateral_minus_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].lateral_minus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].lateral_minus_force_mean)

ValueError: All numbers are identical in kruskal

In [71]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].edge_external_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].edge_external_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].edge_external_force_mean)

KruskalResult(statistic=10.919117647058826, pvalue=0.00425543272753593)

In [72]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.spring == 1].edge_internal_force_mean,
                df_du_27_650.loc[df_du_27_650.spring == 1].edge_internal_force_mean,
                df_kb_27_650.loc[df_kb_27_650.spring == 1].edge_internal_force_mean)

KruskalResult(statistic=4.248529411764714, pvalue=0.1195208189198149)

Масса 27 тонн/ось, кривизна 650, осень

In [98]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].vertical_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].vertical_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].vertical_force_mean)

KruskalResult(statistic=14.866917293233087, pvalue=0.0005911394297641764)

In [ ]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].lateral_plus_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].lateral_plus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].lateral_plus_force_mean)

In [73]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].lateral_minus_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].lateral_minus_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].lateral_minus_force_mean)

KruskalResult(statistic=8.68242819515866, pvalue=0.013020710190351791)

In [74]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].edge_external_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].edge_external_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].edge_external_force_mean)

KruskalResult(statistic=1.1706766917293194, pvalue=0.5569173992322173)

In [75]:
sc.stats.kruskal(df_ars_27_650.loc[df_ars_27_650.fall == 1].edge_internal_force_mean,
                df_du_27_650.loc[df_du_27_650.fall == 1].edge_internal_force_mean,
                df_kb_27_650.loc[df_kb_27_650.fall == 1].edge_internal_force_mean)

KruskalResult(statistic=0.772932330827075, pvalue=0.679453714021163)

Теперь проверим, как масса влияет на силы

Для АРС

In [64]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].vertical_force_mean,
                df_ars_27_0.loc[df_ars_27_0.summer == 1].vertical_force_mean)

KruskalResult(statistic=10.5, pvalue=0.0011937454448720044)

In [66]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_plus_force_mean,
                df_ars_27_0.loc[df_ars_27_0.summer == 1].lateral_plus_force_mean)

KruskalResult(statistic=1.3392857142857224, pvalue=0.24715997321190242)

In [67]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].lateral_minus_force_mean,
                df_ars_27_0.loc[df_ars_27_0.summer == 1].lateral_minus_force_mean)

KruskalResult(statistic=0.4821428571428612, pvalue=0.48745329036403295)

In [69]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_external_force_mean,
                df_ars_27_0.loc[df_ars_27_0.summer == 1].edge_external_force_mean)

KruskalResult(statistic=9.053571428571438, pvalue=0.0026218217690905793)

In [68]:
sc.stats.kruskal(df_ars_23_0.loc[df_ars_23_0.summer == 1].edge_internal_force_mean,
                df_ars_27_0.loc[df_ars_27_0.summer == 1].edge_internal_force_mean)

KruskalResult(statistic=10.5, pvalue=0.0011937454448720044)

## Визуализация данных для вертикальных сил

In [121]:
i = 0
for df_tmp in df_list:

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)


    trace_summer = go.Scatter3d(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
        z=df_tmp.loc[df_tmp.summer == 1].temperature,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter3d(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
        z=df_tmp.loc[df_tmp.fall == 1].temperature,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter3d(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
        z=df_tmp.loc[df_tmp.spring == 1].temperature,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1

## Визуализация данных для положительных боковых сил

In [293]:
i = 0
for df_tmp in df_list:

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)


    trace_summer = go.Scatter3d(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
        z=df_tmp.loc[df_tmp.summer == 1].temperature,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter3d(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
        z=df_tmp.loc[df_tmp.fall == 1].temperature,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter3d(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
        z=df_tmp.loc[df_tmp.spring == 1].temperature,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1

## Визуализация данных для боковых отрицательных сил

In [122]:
i = 0
for df_tmp in df_list:

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)


    trace_summer = go.Scatter3d(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
        z=df_tmp.loc[df_tmp.summer == 1].temperature,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter3d(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
        z=df_tmp.loc[df_tmp.fall == 1].temperature,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter3d(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
        z=df_tmp.loc[df_tmp.spring == 1].temperature,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1

## Визуализация данных для внешней кромки

In [123]:
i = 0
for df_tmp in df_list:

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)


    trace_summer = go.Scatter3d(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
        z=df_tmp.loc[df_tmp.summer == 1].temperature,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter3d(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
        z=df_tmp.loc[df_tmp.fall == 1].temperature,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter3d(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
        z=df_tmp.loc[df_tmp.spring == 1].temperature,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1

## Визуализация данных для внутренней кромочной силы 

## Полиноминальный регрессор для вертикальных сил

In [78]:
i = 0
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    i = 0
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    print(name_list[i])
    spring_np = np.array(df_tmp.loc[df_tmp.spring == 1].vertical_force_mean)
    summer_np = np.array(df_tmp.loc[df_tmp.summer == 1].vertical_force_mean)
    fall_np = np.array(df_tmp.loc[df_tmp.fall == 1].vertical_force_mean)

    if spring_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(spring_np.reshape(-1, 1))
        print('--Весна--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(spring_np)))
    elif spring_np.shape[0] == 1:
        print('--Весна--\nМат. ожидание: ' + str(spring_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(spring_np)))
        
    if summer_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(summer_np.reshape(-1, 1))
        print('--Лето--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(summer_np)))
    elif summer_np.shape[0] == 1:
         print('--Лето--\nМат. ожидание: ' + str(summer_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(summer_np)))
    
    if fall_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(fall_np.reshape(-1, 1))
        print('--Осень--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(fall_np)))
    elif fall_np.shape[0] == 1:
        print('--Осень--\nМат. ожидание: ' + str(fall_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(fall_np)))
        
    print()
    
    ridge1 = Ridge()
    model1 = make_pipeline(PolynomialFeatures(2), ridge1)
    model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.summer == 1].vertical_force_mean))

    trace_summer_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для лета',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )
    
    print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    ridge2 = Ridge()
    model2 = make_pipeline(PolynomialFeatures(2), ridge2)
    model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.fall == 1].vertical_force_mean))

    trace_fall_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для осени',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    ridge3 = Ridge()
    model3 = make_pipeline(PolynomialFeatures(2), ridge3)
    model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
               np.array(df_tmp.loc[df_tmp.spring == 1].vertical_force_mean))

    trace_spring_predict = go.Scatter(
        x=np.linspace(0, 30, 100),
        y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
        mode='lines',
        name='Предсказание для весны',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )
          
    print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    trace_summer = go.Scatter(
        x=df_tmp.loc[df_tmp.summer == 1].speed,
        y=df_tmp.loc[df_tmp.summer == 1].vertical_force_mean,
        mode='markers',
        name='Лето',
        marker = dict(
            size = 12,
            color = '#4F7942',
            line = dict(
                width = 1,
            )
        )
    )

    trace_fall = go.Scatter(
        x=df_tmp.loc[df_tmp.fall == 1].speed,
        y=df_tmp.loc[df_tmp.fall == 1].vertical_force_mean,
        mode='markers',
        name='Осень',
        marker = dict(
            size = 12,
            color = '#FFA500',
            line = dict(
                width = 1,
            )
        )
    )

    trace_spring = go.Scatter(
        x=df_tmp.loc[df_tmp.spring == 1].speed,
        y=df_tmp.loc[df_tmp.spring == 1].vertical_force_mean,
        mode='markers',
        name='Весна',
        marker = dict(
            size = 12,
            color = '#00BFFF',
            line = dict(
                width = 1,
            )
        )
    )

    data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))
    i = i + 1

    fig = dict(data=data, layout=layout)
    iplot(fig)

АРС, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 178781.99999999994
Дисперсия: 14080881.715290619
Количество: 7
--Лето--
Мат. ожидание: 112648.71428571426
Дисперсия: 18253765.633656096
Количество: 7
--Осень--
Мат. ожидание: 104031.28571428568
Дисперсия: 22287613.919365477
Количество: 7

Лето: [138223.45461452272, -2198.115543705481, 35.953390498548046]
Осень: [143463.8371385099, -3934.2409846173387, 90.31897963274464]
Весна: [199768.51340539422, -1807.1818530658634, 34.33839204702935]


АРС, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 268046.1428571428
Дисперсия: 68850016.12346704
Количество: 7
--Лето--
Мат. ожидание: 124938.99999999996
Дисперсия: 9840070.001005722
Количество: 5
--Осень--
Мат. ожидание: 112914.71428571426
Дисперсия: 30088551.9193712
Количество: 7

Лето: [150156.3539459595, -3069.298197149232, 85.94097528222147]
Осень: [146405.38856275278, -2782.1298409003284, 42.11199028332439]
Весна: [228801.12150490977, 5216.321640027109, -163.29763332127453]


АРС, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 203668.49999999994
Дисперсия: 143575379.25101525
Количество: 8
--Лето--
Мат. ожидание: 129361.62499999997
Дисперсия: 14965908.985378815
Количество: 8
--Осень--
Мат. ожидание: 115677.16666666664
Дисперсия: 33601513.473223284
Количество: 6

Лето: [155764.78175195123, -2903.0400797869947, 75.25601709938127]
Осень: [92091.18544722252, 4505.049399162559, -174.22025790853735]
Весна: [146796.39968517073, 7647.997743846789, -244.78850719977984]


АРС, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 280529.0
Дисперсия: 0
Количество: 1
--Лето--
Мат. ожидание: 144446.49999999997
Дисперсия: 130125782.65100153
Количество: 10
--Осень--
Мат. ожидание: 127096.85714285712
Дисперсия: 60826892.40916689
Количество: 7

Лето: [148395.31716338109, 1427.8955662812073, -106.70291000669394]
Осень: [123408.37066815606, 2633.4438769272497, -142.5613255845253]
Весна: [280529.0, 0.0, 0.0]


ДУ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 117743.39999999995
Дисперсия: 592424799.4410025
Количество: 5
--Лето--
Мат. ожидание: 144534.79999999996
Дисперсия: 54927094.96101053
Количество: 5
--Осень--
Мат. ожидание: 103928.74999999997
Дисперсия: 2809634.438503815
Количество: 8

Лето: [129425.12639033391, 70.63528825728473, 63.36657789341498]
Осень: [101666.97827268818, 300.0408580881728, -6.499323312753022]
Весна: [141111.6985091159, 6202.028083114161, -485.00562398012653]


...

ДУ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 133298.79999999996
Дисперсия: 58228635.36101206
Количество: 5
--Лето--
Мат. ожидание: 119423.24999999994
Дисперсия: 80796160.68850763
Количество: 4
--Осень--
Мат. ожидание: 111792.83333333331
Дисперсия: 86036327.80655725
Количество: 6

Лето: [82637.5225935154, 3636.0566142169487, -74.5032470438807]
Осень: [73400.73666506453, 2969.203080579812, 4.734434895924584]
Весна: [112600.06869726695, 25.304074276389223, 92.44989372348145]


КБ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 127330.74999999997
Дисперсия: 1159023060.4385037
Количество: 8
--Лето--
Мат. ожидание: 121674.99999999996
Дисперсия: 20346956.801006865
Количество: 5
--Осень--
Мат. ожидание: 106348.39999999997
Дисперсия: 34905271.04100473
Количество: 5

Лето: [99922.06514682062, 2266.398383543437, -50.377496184650916]
Осень: [98310.23079329584, 2554.055727258937, -136.91834233604195]
Весна: [89229.16848772694, 9520.633476330795, -415.3595786255886]


КБ, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 151112.74999999994
Дисперсия: 6689398.18850763
Количество: 4
--Лето--
Мат. ожидание: 125580.66666666664
Дисперсия: 27480842.223227095
Количество: 6
--Осень--
Мат. ожидание: 110372.00000000001
Дисперсия: 0
Количество: 1

Лето: [90858.05929636017, 4585.128684058846, -143.07955813960243]
Осень: [110372.00000000001, 0.0, 0.0]
Весна: [156576.99412179217, 47.59759130524434, -42.890966388325786]


КБ, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 163354.09090909088
Дисперсия: 88928798.26546152
Количество: 11
--Лето--
Мат. ожидание: 131739.19999999998
Дисперсия: 41919211.36100824
Количество: 10
--Осень--
Мат. ожидание: 123452.3333333333
Дисперсия: 57005773.22322519
Количество: 6

Лето: [151655.15351317546, -4280.021679100129, 186.28593403160158]
Осень: [99150.93126800752, 5572.44276499236, -254.0326001245638]
Весна: [128858.37367992905, 7574.681416257419, -352.2868490747505]


КБ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 164788.69999999995
Дисперсия: 51624306.411003664
Количество: 10
--Лето--
Мат. ожидание: 133176.24999999997
Дисперсия: 103899621.18850382
Количество: 12
--Осень--
Мат. ожидание: 126657.99999999997
Дисперсия: 48709261.33433397
Количество: 6

Лето: [149045.9242994288, -3706.3687852071794, 174.15802222988776]
Осень: [117906.43067892508, 358.39629584100896, 26.067159660995035]
Весна: [164971.71409784234, -2325.3214452806587, 167.005040350552]


## Полимиальный регрессор для боковых положительных сил

In [79]:
i = 0
data = list()
data.clear()
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    print(name_list[i])
    spring_np = np.array(df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean)
    summer_np = np.array(df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean)
    fall_np = np.array(df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean)

    if spring_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(spring_np.reshape(-1, 1))
        print('--Весна--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(spring_np)))
    elif spring_np.shape[0] == 1:
        print('--Весна--\nМат. ожидание: ' + str(spring_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(spring_np)))
        
    if summer_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(summer_np.reshape(-1, 1))
        print('--Лето--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(summer_np)))
    elif summer_np.shape[0] == 1:
         print('--Лето--\nМат. ожидание: ' + str(summer_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(summer_np)))
    
    if fall_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(fall_np.reshape(-1, 1))
        print('--Осень--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(fall_np)))
    elif fall_np.shape[0] == 1:
        print('--Осень--\nМат. ожидание: ' + str(fall_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(fall_np)))
        
    print()
    
    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        ridge1 = Ridge()
        model1 = make_pipeline(PolynomialFeatures(2), ridge1)
        model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean))

        trace_summer_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для лета',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_summer_predict)

        print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        ridge2 = Ridge()
        model2 = make_pipeline(PolynomialFeatures(2), ridge2)
        model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean))

        trace_fall_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для осени',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_fall_predict)

        print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))
    
    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        ridge3 = Ridge()
        model3 = make_pipeline(PolynomialFeatures(2), ridge3)
        model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean))

        trace_spring_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для весны',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_spring_predict)

        print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        trace_summer = go.Scatter(
            x=df_tmp.loc[df_tmp.summer == 1].speed,
            y=df_tmp.loc[df_tmp.summer == 1].lateral_plus_force_mean,
            mode='markers',
            name='Лето',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_summer)
    
    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        trace_fall = go.Scatter(
            x=df_tmp.loc[df_tmp.fall == 1].speed,
            y=df_tmp.loc[df_tmp.fall == 1].lateral_plus_force_mean,
            mode='markers',
            name='Осень',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_fall)

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        trace_spring = go.Scatter(
            x=df_tmp.loc[df_tmp.spring == 1].speed,
            y=df_tmp.loc[df_tmp.spring == 1].lateral_plus_force_mean,
            mode='markers',
            name='Весна',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_spring)

    #data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1
    data.clear()

АРС, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 15110.142857142853
Дисперсия: 14561727.837734712
Количество: 7
--Лето--
Мат. ожидание: 16629.571428571424
Дисперсия: 9459417.960183753
Количество: 7
--Осень--
Мат. ожидание: 14055.999999999996
Дисперсия: 5901641.715285821
Количество: 7

Лето: [12663.754709100846, 234.04909696556422, 0.8366972695299608]
Осень: [-815.2552010511299, 1852.058970723896, -55.198981643624926]
Весна: [19025.245870875835, -1105.7939963565536, 47.17587331221528]


АРС, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 24362.428571428565
Дисперсия: 2894464.245898081
Количество: 7
--Лето--
Мат. ожидание: 22269.39999999999
Дисперсия: 66134477.841000214
Количество: 5
--Осень--
Мат. ожидание: 30915.999999999993
Дисперсия: 103825956.28671451
Количество: 7

Лето: [67084.28327269826, -6327.644618082251, 206.78128233790454]
Осень: [1958.5526812069802, 5601.556979775717, -226.37546817599602]
Весна: [27158.27167611707, -179.58887042777914, 0.43936637150083163]


АРС, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 18426.874999999996
Дисперсия: 15010831.11037506
Количество: 8
--Лето--
Мат. ожидание: 14460.249999999996
Дисперсия: 9094395.68850006
Количество: 8
--Осень--
Мат. ожидание: 15741.666666666662
Дисперсия: 241681.8898889253
Количество: 6

Лето: [9056.799389404572, 1382.2982414944097, -58.03494627330833]
Осень: [16514.37058446521, 19.28721061179872, -3.879236518128923]
Весна: [33930.00287677336, -1648.9997424351284, 40.085270353692906]


АРС, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 24141.0
Дисперсия: 0
Количество: 1
--Лето--
Мат. ожидание: 25624.999999999993
Дисперсия: 12490569.80100019
Количество: 10
--Осень--
Мат. ожидание: 31145.71428571428
Дисперсия: 12275811.633653237
Количество: 7

Лето: [23526.678004435435, -181.56310485789672, 22.285726805938726]
Осень: [45000.796762364305, -1764.7143030310176, 53.18467387291066]
Весна: [24141.0, 0.0, 0.0]


ДУ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 17980.799999999992
Дисперсия: 29332546.961000036
Количество: 5
--Лето--
Мат. ожидание: 29297.599999999988
Дисперсия: 100145692.24100013
Количество: 5
--Осень--
Мат. ожидание: 12660.999999999996
Дисперсия: 20080754.001000058
Количество: 8

Лето: [-1793.2932271849713, 103.15639874086388, 133.2195734290763]
Осень: [14398.583258314984, -520.9668154548459, 26.91390446839156]
Весна: [36069.88211806224, -1820.7351585354488, 41.41447155591427]


...

ДУ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 37206.19999999999
Дисперсия: 17112454.961000513
Количество: 5
--Лето--
Мат. ожидание: 115275.49999999994
Дисперсия: 2181859607.251002
Количество: 4
--Осень--
Мат. ожидание: 28988.66666666666
Дисперсия: 26604352.889888953
Количество: 6

Лето: [-4223.676333187861, 14648.71611793422, -431.82230832263826]
Осень: [10219.169141041864, 2547.1877208724477, -80.20079190517889]
Весна: [48784.59816383186, -313.70906904768805, -31.887004006352687]


КБ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 11929.374999999998
Дисперсия: 23340898.485375058
Количество: 8
--Лето--
Мат. ожидание: 18390.199999999993
Дисперсия: 3935616.961000217
Количество: 5
--Осень--
Мат. ожидание: 13899.199999999995
Дисперсия: 28330420.161000054
Количество: 5

Лето: [14835.033185855053, 368.1644512654166, -8.09680320573066]
Осень: [15244.799516140076, 590.6932059387836, -48.4857366688408]
Весна: [12891.53489879185, 33.90636703199828, -10.06077386834533]


КБ, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 22078.74999999999
Дисперсия: 4680491.188500239
Количество: 4
--Лето--
Мат. ожидание: 27224.499999999993
Дисперсия: 12689503.917666866
Количество: 6
--Осень--
Мат. ожидание: 17793.0
Дисперсия: 0
Количество: 1

Лето: [13597.390514226689, 2361.4099892818986, -91.45710432508663]
Осень: [17793.0, 0.0, 0.0]
Весна: [29174.13647425252, 64.47540858590426, -55.9175318921512]


КБ, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 13292.272727272726
Дисперсия: 3126198.7448016535
Количество: 11
--Лето--
Мат. ожидание: 18183.499999999996
Дисперсия: 10193417.25100006
Количество: 10
--Осень--
Мат. ожидание: 16340.499999999996
Дисперсия: 26483087.58433331
Количество: 6

Лето: [20836.762888760277, -657.6830079766845, 30.437546173466163]
Осень: [19842.28818785709, -197.93105129580425, -3.4202109730307653]
Весна: [-902.9123991209126, 2327.5693734917295, -90.33923734932625]


КБ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 23871.499999999996
Дисперсия: 16683835.05100019
Количество: 10
--Лето--
Мат. ожидание: 27526.249999999993
Дисперсия: 60717435.35516687
Количество: 12
--Осень--
Мат. ожидание: 22742.999999999996
Дисперсия: 137127288.66766667
Количество: 6

Лето: [41560.101156800134, -2717.056578894486, 115.80352317969117]
Осень: [33229.91482780686, -3715.2903282671073, 218.45833515240056]
Весна: [38582.04961958632, -2392.9348758342944, 94.06910236258501]


## Полимиальный регрессор для боковых отрицательных сил

In [82]:
i = 0
data = list()
data.clear()
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    print(name_list[i])
    spring_np = np.array(df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean)
    summer_np = np.array(df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean)
    fall_np = np.array(df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean)

    if spring_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(spring_np.reshape(-1, 1))
        print('--Весна--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(spring_np)))
    elif spring_np.shape[0] == 1:
        print('--Весна--\nМат. ожидание: ' + str(spring_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(spring_np)))
        
    if summer_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(summer_np.reshape(-1, 1))
        print('--Лето--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(summer_np)))
    elif summer_np.shape[0] == 1:
         print('--Лето--\nМат. ожидание: ' + str(summer_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(summer_np)))
    
    if fall_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(fall_np.reshape(-1, 1))
        print('--Осень--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(fall_np)))
    elif fall_np.shape[0] == 1:
        print('--Осень--\nМат. ожидание: ' + str(fall_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(fall_np)))
        
    print()
    
    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        ridge1 = Ridge()
        model1 = make_pipeline(PolynomialFeatures(2), ridge1)
        model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean))

        trace_summer_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для лета',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_summer_predict)

        print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        ridge2 = Ridge()
        model2 = make_pipeline(PolynomialFeatures(2), ridge2)
        model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean))

        trace_fall_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для осени',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )

        data.append(trace_fall_predict)
        
        print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        ridge3 = Ridge()
        model3 = make_pipeline(PolynomialFeatures(2), ridge3)
        model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean))

        trace_spring_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для весны',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )

        data.append(trace_spring_predict)
        
        print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))
                
    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        trace_summer = go.Scatter(
            x=df_tmp.loc[df_tmp.summer == 1].speed,
            y=df_tmp.loc[df_tmp.summer == 1].lateral_minus_force_mean,
            mode='markers',
            name='Лето',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_summer)

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        trace_fall = go.Scatter(
            x=df_tmp.loc[df_tmp.fall == 1].speed,
            y=df_tmp.loc[df_tmp.fall == 1].lateral_minus_force_mean,
            mode='markers',
            name='Осень',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_fall)

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        trace_spring = go.Scatter(
            x=df_tmp.loc[df_tmp.spring == 1].speed,
            y=df_tmp.loc[df_tmp.spring == 1].lateral_minus_force_mean,
            mode='markers',
            name='Весна',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_spring)

    #data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1
    data.clear()

АРС, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: -13972.28571428571
Дисперсия: 23315694.77651028
Количество: 7
--Лето--
Мат. ожидание: -14812.28571428571
Дисперсия: 92959514.49079596
Количество: 7
--Осень--
Мат. ожидание: -37590.85714285713
Дисперсия: 2244114576.694877
Количество: 7

Лето: [18704.40658522314, -4551.35534375958, 147.4106015357701]
Осень: [77940.3269451112, -12364.884783937476, 312.71933505263524]
Весна: [6255.6199472979715, -2046.8922013123617, 49.27633719080781]


АРС, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 0.0
Дисперсия: 0.001
Количество: 7
--Лето--
Мат. ожидание: -25761.799999999992
Дисперсия: 119622468.16100009
Количество: 5
--Осень--
Мат. ожидание: -23764.571428571424
Дисперсия: 235630304.24589795
Количество: 7

Лето: [7924.894197065882, -5528.508014670633, 203.23643352165516]
Осень: [-53480.868368699455, 7449.855486262856, -333.4414749934657]
Весна: [0.0, 0.0, 0.0]


АРС, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: -11304.749999999998
Дисперсия: 3328628.938500015
Количество: 8
--Лето--
Мат. ожидание: -10463.249999999998
Дисперсия: 35111806.43850002
Количество: 8
--Осень--
Мат. ожидание: -4267.333333333332
Дисперсия: 257378.2232222276
Количество: 6

Лето: [-22321.262539804935, 2153.249404863819, -79.74625211937831]
Осень: [-7604.2980594168375, 389.39649670068053, -10.402633483540004]
Весна: [-24091.661093106766, 1646.9623948812887, -50.5999340856484]


АРС, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 0.0
Дисперсия: 0
Количество: 1
--Лето--
Мат. ожидание: -9296.199999999997
Дисперсия: 27331143.96100002
Количество: 10
--Осень--
Мат. ожидание: -10607.428571428569
Дисперсия: 2848835.6744693797
Количество: 7

Лето: [-7377.92207972189, 592.7122781750819, -45.81643346243622]
Осень: [-7923.732218461486, -162.3950109056752, -0.34638207817931416]
Весна: [0.0, 0.0, 0.0]


ДУ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: -8088.399999999997
Дисперсия: 46640240.240999974
Количество: 5
--Лето--
Мат. ожидание: -11755.999999999996
Дисперсия: 19553707.201000016
Количество: 5
--Осень--
Мат. ожидание: -10145.749999999998
Дисперсия: 15386031.43850003
Количество: 8

Лето: [-23195.14272950233, -45.08067001549267, 54.587526384439236]
Осень: [-10236.424714662693, 491.2696365314951, -36.4281937419044]
Весна: [-2499.2397548493955, -2542.3436119583794, 137.428477202616]


...

ДУ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 0.0
Дисперсия: 0.001
Количество: 5
--Лето--
Мат. ожидание: -71778.24999999997
Дисперсия: 5585186608.1885
Количество: 4
--Осень--
Мат. ожидание: -4049.999999999999
Дисперсия: 4807176.334333332
Количество: 6

Лето: [106574.43403976824, -30833.544394009274, 1244.6333116149935]
Осень: [-3143.25435896247, -41.360955352197166, -2.2783686988051466]
Весна: [0.0, 0.0, 0.0]


КБ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: -7429.874999999998
Дисперсия: 37740222.36037502
Количество: 8
--Лето--
Мат. ожидание: -10291.399999999996
Дисперсия: 520377.04100003635
Количество: 5
--Осень--
Мат. ожидание: -14152.999999999995
Дисперсия: 10261033.201000107
Количество: 5

Лето: [-12282.971870345076, 433.1933905746147, -18.362936215300007]
Осень: [-17486.179291473116, 1041.1035895728246, -55.51462749941834]
Весна: [-2975.078330744197, 646.6654551326283, -83.26297609229653]


КБ, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 0.0
Дисперсия: 0.001
Количество: 4
--Лето--
Мат. ожидание: -10784.83333333333
Дисперсия: 13143514.139888896
Количество: 6
--Осень--
Мат. ожидание: -33022.0
Дисперсия: 0
Количество: 1

Лето: [-6.197281525748622, -1769.378112583602, 66.15613232745832]
Осень: [-33022.0, 0.0, 0.0]
Весна: [0.0, 0.0, 0.0]


КБ, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: -7270.090909090908
Дисперсия: 35771600.99273555
Количество: 11
--Лето--
Мат. ожидание: -10388.699999999997
Дисперсия: 12424046.61100003
Количество: 10
--Осень--
Мат. ожидание: -7379.166666666665
Дисперсия: 38766637.47322222
Количество: 6

Лето: [-1284.8947380261543, -661.5310570719118, 1.969626234930113]
Осень: [-18744.331486421564, 2053.9455118563487, -82.27840062761626]
Весна: [-396.3990406905523, 576.6519847486016, -74.14976995339214]


КБ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 0.0
Дисперсия: 0.001
Количество: 10
--Лето--
Мат. ожидание: -15101.666666666662
Дисперсия: 12521509.556555642
Количество: 12
--Осень--
Мат. ожидание: -8616.499999999998
Дисперсия: 19402097.91766667
Количество: 6

Лето: [-5273.24226444819, -1972.237329023605, 85.83061062174527]
Осень: [-11731.1137332951, 1522.6653673109, -96.73951520209977]
Весна: [0.0, 0.0, 0.0]


## Полимиальный регрессор для кромочных внешних сил

In [80]:
i = 0
data = list()
data.clear()
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    print(name_list[i])
    spring_np = np.array(df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean)
    summer_np = np.array(df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean)
    fall_np = np.array(df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean)

    if spring_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(spring_np.reshape(-1, 1))
        print('--Весна--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(spring_np)))
    elif spring_np.shape[0] == 1:
        print('--Весна--\nМат. ожидание: ' + str(spring_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(spring_np)))
        
    if summer_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(summer_np.reshape(-1, 1))
        print('--Лето--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(summer_np)))
    elif summer_np.shape[0] == 1:
         print('--Лето--\nМат. ожидание: ' + str(summer_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(summer_np)))
    
    if fall_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(fall_np.reshape(-1, 1))
        print('--Осень--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(fall_np)))
    elif fall_np.shape[0] == 1:
        print('--Осень--\nМат. ожидание: ' + str(fall_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(fall_np)))
        
    print()
    
    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        ridge1 = Ridge()
        model1 = make_pipeline(PolynomialFeatures(2), ridge1)
        model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean))

        trace_summer_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для лета',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_summer_predict)

        print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        ridge2 = Ridge()
        model2 = make_pipeline(PolynomialFeatures(2), ridge2)
        model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean))

        trace_fall_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для осени',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_fall_predict)

        print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        ridge3 = Ridge()
        model3 = make_pipeline(PolynomialFeatures(2), ridge3)
        model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean))

        trace_spring_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для весны',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        
        data.append(trace_spring_predict)

        print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        trace_summer = go.Scatter(
            x=df_tmp.loc[df_tmp.summer == 1].speed,
            y=df_tmp.loc[df_tmp.summer == 1].edge_external_force_mean,
            mode='markers',
            name='Лето',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_summer)

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        trace_fall = go.Scatter(
            x=df_tmp.loc[df_tmp.fall == 1].speed,
            y=df_tmp.loc[df_tmp.fall == 1].edge_external_force_mean,
            mode='markers',
            name='Осень',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_fall)

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        trace_spring = go.Scatter(
            x=df_tmp.loc[df_tmp.spring == 1].speed,
            y=df_tmp.loc[df_tmp.spring == 1].edge_external_force_mean,
            mode='markers',
            name='Весна',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_spring)

    #data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1
    data.clear()

АРС, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 47978.999999999985
Дисперсия: 332505.7152868042
Количество: 7
--Лето--
Мат. ожидание: 51843.2857142857
Дисперсия: 3755284.4907975923
Количество: 7
--Осень--
Мат. ожидание: 50925.85714285713
Дисперсия: 7975554.4091640245
Количество: 7

Лето: [64649.75360366315, -1161.843545807531, 21.674177569960897]
Осень: [69983.16472088578, -1932.240444992238, 45.421403012058576]
Весна: [47015.38276752603, 59.47193192494751, -0.3298794222279585]


АРС, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 50664.42857142856
Дисперсия: 11446403.96018417
Количество: 7
--Лето--
Мат. ожидание: 64687.799999999974
Дисперсия: 13280442.161000801
Количество: 5
--Осень--
Мат. ожидание: 63542.14285714284
Дисперсия: 34377732.12345083
Количество: 7

Лето: [93333.13920099323, -3413.7705148255704, 93.11891466153456]
Осень: [77478.51807823875, -255.28311165067933, -36.114849087957666]
Весна: [51578.532751713225, 801.6576698982988, -50.01161655674938]


АРС, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 52402.874999999985
Дисперсия: 4772600.860376431
Количество: 8
--Лето--
Мат. ожидание: 56456.124999999985
Дисперсия: 3928232.860375954
Количество: 8
--Осень--
Мат. ожидание: 53008.16666666666
Дисперсия: 5895477.806555821
Количество: 6

Лето: [50323.04708447034, 997.8357244486098, -34.979341385112974]
Осень: [34999.381757714335, 2707.1109600999657, -90.93323020848558]
Весна: [53333.116844724296, -142.44949580514609, 5.064811705848078]


АРС, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 41369.0
Дисперсия: 0
Количество: 1
--Лето--
Мат. ожидание: 65990.59999999999
Дисперсия: 466086510.841003
Количество: 10
--Осень--
Мат. ожидание: 77870.2857142857
Дисперсия: 40157914.20508249
Количество: 7

Лето: [57085.35661549362, 875.0971287657213, -3.538618363425131]
Осень: [85125.19344995297, 813.2721823283894, -75.25353612914702]
Весна: [41369.0, 0.0, 0.0]


ДУ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 70800.19999999997
Дисперсия: 1246377881.7610002
Количество: 5
--Лето--
Мат. ожидание: 23204.999999999993
Дисперсия: 284025507.20100003
Количество: 5
--Осень--
Мат. ожидание: 53185.624999999985
Дисперсия: 3161198.485375954
Количество: 8

Лето: [42031.51063728216, -318.56764119138865, -63.48106768742572]
Осень: [50898.038315587015, 72.93081963555427, 10.824613893423079]
Весна: [-29083.26430704641, 7878.830085803366, -91.76943788366142]


...

ДУ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 74816.99999999997
Дисперсия: 29358025.601002287
Количество: 5
--Лето--
Мат. ожидание: 71781.99999999997
Дисперсия: 25269133.501001906
Количество: 4
--Осень--
Мат. ожидание: 74480.16666666666
Дисперсия: 249185262.1398893
Количество: 6

Лето: [63756.75549965199, 2070.584352808514, -101.71089343585804]
Осень: [-26606.506066359012, 11456.920310783673, -261.61167820676815]
Весна: [90861.14175856739, -667.4015177465922, -28.783668380110928]


КБ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 4204.624999999999
Дисперсия: 3443286.485375004
Количество: 8
--Лето--
Мат. ожидание: 50398.19999999998
Дисперсия: 24593201.361001562
Количество: 5
--Осень--
Мат. ожидание: 41031.39999999999
Дисперсия: 9504569.041000439
Количество: 5

Лето: [24036.686530012492, 2835.602122622065, -66.47558023994124]
Осень: [29889.87305482285, 2371.4389407172384, -107.82390513925326]
Весна: [4544.398270048104, -98.83936264520311, 4.748122593914839]


КБ, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 68434.49999999997
Дисперсия: 22605864.751001906
Количество: 4
--Лето--
Мат. ожидание: 63037.83333333332
Дисперсия: 67788619.13989027
Количество: 6
--Осень--
Мат. ожидание: 59325.0
Дисперсия: 0
Количество: 1

Лето: [64743.61308264475, -1449.7969730817417, 83.96182275676219]
Осень: [59325.0, 0.0, 0.0]
Весна: [80718.41246598154, 436.04001140513367, -123.92125354424977]


КБ, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 3287.272727272727
Дисперсия: 9832438.74480165
Количество: 11
--Лето--
Мат. ожидание: 52257.19999999999
Дисперсия: 40406136.76100071
Количество: 10
--Осень--
Мат. ожидание: 46400.99999999999
Дисперсия: 69220165.00100096
Количество: 6

Лето: [40191.96420644167, 613.1684408471732, 14.320178639374731]
Осень: [47911.31208040009, 1401.920710694081, -99.86396862132418]
Весна: [19027.251545973046, -2249.1435243194614, 77.21574773301468]


КБ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 58748.999999999985
Дисперсия: 14818012.401000096
Количество: 10
--Лето--
Мат. ожидание: 73251.66666666666
Дисперсия: 138456548.05655628
Количество: 12
--Осень--
Мат. ожидание: 74447.83333333331
Дисперсия: 8769316.806557251
Количество: 6

Лето: [65201.932682351384, 1775.9090103153758, -81.24376179997897]
Осень: [86759.51366668669, -1808.3595653470782, 62.434289888439395]
Весна: [51834.718120690784, 642.8550005242594, -9.402884561418784]


## Полимиальный регрессор для кромочных внутренних сил

In [83]:
i = 0
data = list()
data.clear()
for df_tmp in df_list:
    #df_tmp = df_ars_23_0
    
    print(name_list[i])
    spring_np = np.array(df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean)
    summer_np = np.array(df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean)
    fall_np = np.array(df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean)

    if spring_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(spring_np.reshape(-1, 1))
        print('--Весна--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(spring_np)))
    elif spring_np.shape[0] == 1:
        print('--Весна--\nМат. ожидание: ' + str(spring_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(spring_np)))
        
    if summer_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(summer_np.reshape(-1, 1))
        print('--Лето--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(summer_np)))
    elif summer_np.shape[0] == 1:
         print('--Лето--\nМат. ожидание: ' + str(summer_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(summer_np)))
    
    if fall_np.shape[0] > 1:
        gmm = mixture.GMM(n_components=1)
        gmm.fit(fall_np.reshape(-1, 1))
        print('--Осень--\nМат. ожидание: ' + str(gmm.means_[0][0]) + '\nДисперсия: ' + str(gmm.covars_[0][0]) + '\nКоличество: ' + str(len(fall_np)))
    elif fall_np.shape[0] == 1:
        print('--Осень--\nМат. ожидание: ' + str(fall_np[0]) + '\nДисперсия: ' + str(0) + '\nКоличество: ' + str(len(fall_np)))
        
    print()
    
    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        ridge1 = Ridge()
        model1 = make_pipeline(PolynomialFeatures(2), ridge1)
        model1.fit(np.array(df_tmp.loc[df_tmp.summer == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean))

        trace_summer_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model1.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для лета',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )

        data.append(trace_summer_predict)

        print('Лето: ' + str([ridge1.intercept_, ridge1.coef_[1], ridge1.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        ridge2 = Ridge()
        model2 = make_pipeline(PolynomialFeatures(2), ridge2)
        model2.fit(np.array(df_tmp.loc[df_tmp.fall == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean))

        trace_fall_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model2.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для осени',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )

        data.append(trace_fall_predict)

        print('Осень: ' + str([ridge2.intercept_, ridge2.coef_[1], ridge2.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        ridge3 = Ridge()
        model3 = make_pipeline(PolynomialFeatures(2), ridge3)
        model3.fit(np.array(df_tmp.loc[df_tmp.spring == 1].speed).reshape(-1, 1),
                   np.array(df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean))

        trace_spring_predict = go.Scatter(
            x=np.linspace(0, 30, 100),
            y=model3.predict(np.linspace(0, 30, 100).reshape(-1, 1)),
            mode='lines',
            name='Предсказание для весны',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )

        data.append(trace_spring_predict)

        print('Весна: ' + str([ridge3.intercept_, ridge3.coef_[1], ridge3.coef_[2]]))

    if np.array(df_tmp.loc[df_tmp.summer == 1]).shape[0] != 0:
        trace_summer = go.Scatter(
            x=df_tmp.loc[df_tmp.summer == 1].speed,
            y=df_tmp.loc[df_tmp.summer == 1].edge_internal_force_mean,
            mode='markers',
            name='Лето',
            marker = dict(
                size = 12,
                color = '#4F7942',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_summer)

    if np.array(df_tmp.loc[df_tmp.fall == 1]).shape[0] != 0:
        trace_fall = go.Scatter(
            x=df_tmp.loc[df_tmp.fall == 1].speed,
            y=df_tmp.loc[df_tmp.fall == 1].edge_internal_force_mean,
            mode='markers',
            name='Осень',
            marker = dict(
                size = 12,
                color = '#FFA500',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_fall)

    if np.array(df_tmp.loc[df_tmp.spring == 1]).shape[0] != 0:
        trace_spring = go.Scatter(
            x=df_tmp.loc[df_tmp.spring == 1].speed,
            y=df_tmp.loc[df_tmp.spring == 1].edge_internal_force_mean,
            mode='markers',
            name='Весна',
            marker = dict(
                size = 12,
                color = '#00BFFF',
                line = dict(
                    width = 1,
                )
            )
        )
        data.append(trace_spring)

    #data = [trace_summer_predict, trace_fall_predict, trace_spring_predict, trace_summer, trace_fall, trace_spring]

    layout = dict(title = name_list[i],
                  yaxis = dict(title='Сила'),
                  xaxis = dict(title='Скорость'))

    fig = dict(data=data, layout=layout)
    iplot(fig)
    i = i + 1
    data.clear()

АРС, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 44107.85714285713
Дисперсия: 2653342.9805923235
Количество: 7
--Лето--
Мат. ожидание: 40399.85714285713
Дисперсия: 1711879855.2663064
Количество: 7
--Осень--
Мат. ожидание: 51317.2857142857
Дисперсия: 3928014.205082966
Количество: 7

Лето: [-102334.23712664518, 11491.919645078886, -154.07952948033252]
Осень: [64404.81396520598, -1241.2461547985488, 26.27435952024512]
Весна: [45163.038273996426, 172.0557796644736, -12.218830375255425]


АРС, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 38999.428571428565
Дисперсия: 3178119.960183693
Количество: 7
--Лето--
Мат. ожидание: 45575.79999999998
Дисперсия: 3384074.561001135
Количество: 5
--Осень--
Мат. ожидание: 46032.2857142857
Дисперсия: 18931117.062224936
Количество: 7

Лето: [34600.41762689325, 1162.2890740058142, -26.658611602314267]
Осень: [47882.41062730612, 40.81802515043399, -9.234915523087357]
Весна: [27884.513523331167, 1216.4359684783617, -31.03849781295654]


АРС, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 50825.499999999985
Дисперсия: 6598289.501000954
Количество: 8
--Лето--
Мат. ожидание: 67522.87499999999
Дисперсия: 3402790.360375954
Количество: 8
--Осень--
Мат. ожидание: 62009.33333333332
Дисперсия: 2171187.5565562975
Количество: 6

Лето: [81068.26635619177, -1542.5189708107994, 41.485413165284584]
Осень: [51975.360988267785, 1539.1716211518853, -52.4375279842492]
Весна: [34446.187260230494, 2166.1259637037483, -68.26728207127393]


АРС, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 41385.0
Дисперсия: 0
Количество: 1
--Лето--
Мат. ожидание: 38229.899999999994
Дисперсия: 330147972.491
Количество: 10
--Осень--
Мат. ожидание: 42293.14285714285
Дисперсия: 4113301.266305996
Количество: 7

Лето: [33512.6707365348, -1223.9057677502847, 98.73719047402726]
Осень: [23484.7279349372, 2451.1982509956715, -75.49719950374964]
Весна: [41385.0, 0.0, 0.0]


ДУ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 38761.79999999999
Дисперсия: 120314645.36100037
Количество: 5
--Лето--
Мат. ожидание: 65259.99999999998
Дисперсия: 33389587.201001238
Количество: 5
--Осень--
Мат. ожидание: 73877.12499999999
Дисперсия: 9454613.360375954
Количество: 8

Лето: [102541.32415527545, 101.24664517025698, -174.8408854008148]
Осень: [77449.36009448419, -475.55514192021366, 10.391161045545992]
Весна: [66370.18978173634, -1170.901288793045, -38.01868340951317]


...

ДУ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 56553.59999999998
Дисперсия: 12752725.84100063
Количество: 5
--Лето--
Мат. ожидание: 36740.749999999985
Дисперсия: 41542282.68850072
Количество: 4
--Осень--
Мат. ожидание: 42110.16666666666
Дисперсия: 35526363.139889315
Количество: 6

Лето: [11683.014423968176, 2723.2185851237177, -67.2354193001806]
Осень: [66358.1672326352, -3544.3121276698153, 122.71367515587671]
Весна: [48131.961564152785, 4.354373741242625, 38.00807897053751]


КБ, 23.5 тонн/ось, прямая
--Весна--
Мат. ожидание: 46922.37499999999
Дисперсия: 153789806.98537594
Количество: 8
--Лето--
Мат. ожидание: 60837.39999999998
Дисперсия: 10675938.241000248
Количество: 5
--Осень--
Мат. ожидание: 54008.79999999998
Дисперсия: 24596292.1610008
Количество: 5

Лето: [75364.28223912785, -2227.9999676274474, 77.17244893014632]
Осень: [31705.191490392004, 3596.071354365511, -135.11833529914438]
Весна: [34316.6568968597, 3111.795873249295, -134.568721858181]


КБ, 23.5 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 37471.749999999985
Дисперсия: 2464026.688500477
Количество: 4
--Лето--
Мат. ожидание: 44691.99999999999
Дисперсия: 42588768.66766651
Количество: 6
--Осень--
Мат. ожидание: 36569.0
Дисперсия: 0
Количество: 1

Лето: [24718.292185506907, 1807.6635590701758, -30.16758011368323]
Осень: [36569.0, 0.0, 0.0]
Весна: [36434.986518008605, -378.63105949169113, 39.02808647588144]


КБ, 27 тонн/ось, прямая
--Весна--
Мат. ожидание: 62992.090909090904
Дисперсия: 5313149.901827408
Количество: 11
--Лето--
Мат. ожидание: 75323.89999999998
Дисперсия: 13449490.891001297
Количество: 10
--Осень--
Мат. ожидание: 67322.16666666666
Дисперсия: 7897965.139889313
Количество: 6

Лето: [89642.14381073372, -2127.4694968207336, 72.92724292488977]
Осень: [65415.32156999543, 1033.6024225318372, -59.38366473288691]
Весна: [46817.866238627365, 2646.2311521845004, -102.52993210561543]


КБ, 27 тонн/ось, радиус 650
--Весна--
Мат. ожидание: 55475.19999999999
Дисперсия: 36006657.16100128
Количество: 10
--Лето--
Мат. ожидание: 34278.74999999999
Дисперсия: 327646521.5218335
Количество: 12
--Осень--
Мат. ожидание: 39453.16666666666
Дисперсия: 33822896.80655606
Количество: 6

Лето: [57023.166357276554, -7087.326416256181, 370.6125551999094]
Осень: [23236.175673835372, 1693.7764439428768, -29.94134886610805]
Весна: [54231.295433654755, -553.957699602977, 46.68219268052394]
